# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = os.path.join("..","WeatherPy","City_Weather.csv")
cityDF=pd.read_csv(file)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cityDF[["Lat", "Lng"]]
humidity = cityDF["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowDF=cityDF[cityDF["Humidity"]<40]
narrowDF=narrowDF[narrowDF["Max Temp"]<79]
narrowDF=narrowDF[narrowDF["Max Temp"]>70]
narrowDF=narrowDF[narrowDF["Cloudiness"]<70]
narrowDF

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
10,bam,0.0,ir,1.600283e+09,18.0,29.106000,58.357000,74.14,13.35
45,oshakati,0.0,na,1.600283e+09,11.0,-17.783333,15.683333,73.40,2.24
110,indiana,1.0,us,1.600283e+09,36.0,40.621389,-79.152778,78.01,4.70
115,dumesti,0.0,ro,1.600283e+09,31.0,46.850000,27.283333,78.80,5.82
120,poland,1.0,us,1.600283e+09,38.0,44.060556,-70.394167,73.99,17.22
212,samaipata,0.0,bo,1.600283e+09,33.0,-18.150000,-63.866667,75.90,11.97
304,comarapa,0.0,bo,1.600283e+09,35.0,-17.900000,-64.483333,75.69,11.18
372,vostan,0.0,am,1.600283e+09,34.0,39.958889,44.559167,78.80,16.11
390,qarchak,0.0,ir,1.600283e+09,29.0,35.439444,51.568889,75.20,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=pd.DataFrame()
for index,row in narrowDF.iterrows():
    
    target_location = str(row["Lat"])+","+str(row["Lng"])
    target_radius = 5000
    target_type = "lodging"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": target_location,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    try:
        hotelName=response["results"][0]["name"]
    except IndexError:
        continue
    lat=response["results"][0]["geometry"]["location"]["lat"]
    lng=response["results"][0]["geometry"]["location"]["lng"]
    hotel_df=hotel_df.append({"Hotel Name":hotelName,"City":row["City"],"Country":row["Country"],"Lat":lat,"Lng":lng},ignore_index=True)
    
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,bam,ir,Bam Tourist Hotel,29.095207,58.356684
1,oshakati,na,Oshakati Guest Hotel,-17.794271,15.701723
2,indiana,us,Hampton Inn Indiana,40.602813,-79.181535
3,dumesti,ro,La trăistărie,46.828770,27.239363
4,poland,us,Poland Spring Resort,44.031388,-70.367463
5,samaipata,bo,Casa - Hostal,-18.177030,-63.873274
6,vostan,am,Target Shooting Club,39.945022,44.612846
7,qarchak,ir,Mehdi Ziaee trait,35.438361,51.570334


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,hover_text=hotel_info)

fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))